<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_Output_Parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will learn how to use LangChain's output parser to process LLM output in a more programming language friendly way.

In [38]:
!pip install langchain openai -qU

# CommaSeparatedListOutputParser

In [39]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [37]:
OPENAI_API_KEY = 'your openai api key here'

In [46]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List 3 main-stream {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

In [47]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [48]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [52]:
_input = prompt.format(subject="music styles")
output = llm(_input)


In [53]:
print(output)



Pop, Rock, Hip-Hop


In [54]:
output_parser.parse(output)

['Pop', 'Rock', 'Hip-Hop']

# EnumOutputParser

In [55]:
from langchain.output_parsers.enum import EnumOutputParser

In [56]:
from enum import Enum

class Genders(Enum):
    MALE = "male"
    FEMALE = "female"

In [57]:
output_parser = EnumOutputParser(enum=Genders)
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="Tell me the gender of the celebrity {name}.\n{format_instructions}",
    input_variables=["name"],
    partial_variables={"format_instructions": format_instructions}
)

In [58]:
print(format_instructions)

Select one of the following options: male, female


In [59]:
_input = prompt.format(name="Michael Jordan")
output = llm(_input)
print(output)


male


In [60]:
output_parser.parse(output)

<Genders.MALE: 'male'>

In [61]:
output_parser.parse('xyz')

OutputParserException: ignored

# StructuredOutputParser

In [62]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [63]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the human's question"),
    ResponseSchema(name="source", description="source used to answer the human's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [64]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [65]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // answer to the human's question
	"source": string  // source used to answer the human's question, should be a website.
}
```


In [66]:
_input = prompt.format_prompt(question="what are the ingredients of milk?")
output = llm(_input.to_string())

print(output)



```json
{
	"answer": "Milk is typically made up of water, fat, proteins, lactose (sugar) and minerals.",
	"source": "https://www.dairynz.co.nz/nutrition/what-is-in-milk/"
}
```


In [68]:
json_data = output_parser.parse(output)

In [69]:
print(json_data['answer'])

Milk is typically made up of water, fat, proteins, lactose (sugar) and minerals.
